In [3]:
!pip install sqlalchemy==1.3.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 94.5 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=5b09b41f128b127e66665624208758db803fd99b4602e3d28c25cd3fedcb31c8
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


In [4]:
%load_ext sql

In [5]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [6]:
!pip install -q pandas==1.1.5

In [7]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [8]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


In [9]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [10]:
%sql select distinct Launch_Site from SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [11]:
%sql select * from SPACEXTBL where Launch_Site like 'CCA%' limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [12]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where Customer like 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


In [13]:
%sql select AVG(PAYLOAD_MASS__KG_) from SPACEXTBL where Booster_Version like'F9 v1.1'

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2928.4


In [14]:
%sql select min(Date) from SPACEXTBL where Landing_Outcome like'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


min(Date)
2015-12-22


In [15]:
%sql select distinct Booster_Version from SPACEXTBL where Landing_Outcome like 'Success%(drone_ship)' and PAYLOAD_MASS__KG_ between 4000 and 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [21]:
%sql select count (Mission_Outcome) AS Successful_Outcome, (select count (Mission_Outcome) from SPACEXTBL where Mission_Outcome like 'Failure%') AS Failure_Outcome  from SPACEXTBL where Mission_Outcome like 'Success%'

 * sqlite:///my_data1.db
Done.


Successful_Outcome,Failure_Outcome
100,1


In [22]:
%sql select Booster_Version from SPACEXTBL where PAYLOAD_MASS__KG_ == (select max(PAYLOAD_MASS__KG_) from SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [23]:
%sql select substr(Date, 6,2) as Month, Booster_Version, Landing_Outcome, Launch_site  from SPACEXTBL where Landing_Outcome like 'Failure (drone ship)' and substr(Date,0,5)='2015'

 * sqlite:///my_data1.db
Done.


Month,Booster_Version,Landing_Outcome,Launch_Site
01,F9 v1.1 B1012,Failure (drone ship),CCAFS LC-40
04,F9 v1.1 B1015,Failure (drone ship),CCAFS LC-40


In [17]:
%sql select Landing_Outcome, count(Landing_Outcome) from SPACEXTBL  where Date between '2010-06-04' and '2017-03-20' group by Landing_Outcome order by count(Landing_Outcome) desc

 * sqlite:///my_data1.db
Done.


Landing_Outcome,count(Landing_Outcome)
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
